# spectra extraction and psf matching

the following codes extract the line maps from the existing data products and performs psf matching
the extracted files are saved following the prefix .extract.fits 
the entries of each fits files:

0 PRIMARY

1 ZFIT_STACK

2 SEG

3 DSCI

4 LINE_HA

5 LINEWHT_HA

6 LINE_HB

7 LINEWHT_HB

8 PSF_HA

9 PSF_HB

10 PSF_MATCH

11 LINE_HB_CONVOLVED

12 LINWHT_HB_CONVOLVED

In [1]:
from    astropy.table       import Table
import  numpy               as     np
from    astropy.io          import fits
import  webbpsf
from    photutils.psf       import matching as match
from    astropy.convolution import convolve_fft 
from    tqdm                import tqdm
import  os
import  gc                                         


#this deals with warnings
import warnings                                   
from astropy.io.fits.verify import VerifyWarning                          
warnings.simplefilter('ignore', VerifyWarning)
warnings.simplefilter('ignore', UserWarning) 
warnings.filterwarnings('ignore', category=RuntimeWarning)

#this is just a handy little function to return the desired file path
#give one entry in the objectlist, return the desired file path
def file_path(obj,prefix,filetype='fits'):
    if   filetype == 'fits':
        return f"data/{obj['field']}/{obj['field']}_{str(obj['id']).zfill(5)}.{prefix}.{filetype}"
    elif filetype == 'png':
        return f"png/{obj['field']}/{obj['field']}_{str(obj['id']).zfill(5)}.{prefix}.{filetype}"

### extract ha hb line maps from full.fits data

In [2]:
def exrtract_HaHb(hdu):
    """

    pass objs from obj_lis to extract ha hb lines

    return: HDUlist with the following entry:

    0 primary extension, same as original file

    1 line-fit results

    2 segmentation map

    3 clear filter maps

    4,5 Ha line map & line weight

    6,7 Hb line map & line weight


    """
    #set up a crop of 50x50 pix in the center
    center_size = 50; shape = hdu[5].shape[0]
    #start index: si and end index: ei
    si = (shape - center_size) // 2; 
    ei = si + center_size


    new_file = fits.HDUList()
    #save primary extension
    new_file.append(hdu[0])
    #save line-fit info
    new_file.append(hdu[1])
    """
    select segmentation map[4]
    also save 1 DSCI image for comparison [5]
    """
    
    for i in [4,5]: 
        hdu[i].data = hdu[i].data[si:ei,si:ei]
        new_file.append(hdu[i])

    #loop to select ha hb line maps
    for image in hdu:
        if image.ver in ['Ha','Hb'] and (image.name == 'LINE' or image.name == 'LINEWHT'):
            image.data = image.data[si:ei,si:ei]
            image.name = f'{image.name}_{image.ver}'
            new_file.append(image)
    return new_file


### calculate monochromatic PSFs for Ha Hb line maps, generating kernels for Hb psf matching

In [3]:
# filter_data
NIRISS_filters = [
    {"name": "F090W", "lambda_min": 0.796, "lambda_max": 1.005},
    {"name": "F115W", "lambda_min": 1.013, "lambda_max": 1.283},
    {"name": "F150W", "lambda_min": 1.330, "lambda_max": 1.671},
    {"name": "F200W", "lambda_min": 1.751, "lambda_max": 2.226},
    {"name": "F277W", "lambda_min": 2.413, "lambda_max": 3.143},
    {"name": "F356W", "lambda_min": 3.140, "lambda_max": 4.068},
    {"name": "F444W", "lambda_min": 3.880, "lambda_max": 5.023}
]

#calculate mono psf
def mono_webbpsf(wavelength,filter):
    niriss = webbpsf.NIRISS()
    niriss.filter = filter
    return niriss.calc_psf(fov_pixels=20,monochromatic=wavelength)[3]

def calc_psf(z):
    #use redshift to get line wave length
    waves = [6.563e-7 *(1+z), 4.861e-7 *(1+z)]
    filters = []
    #locate filter for Ha Hb linemaps
    for wave in waves:
        for f in NIRISS_filters:
            if f["lambda_min"] <= wave*1e6 <= f["lambda_max"]:
                filters.append( f"{f['name']}")
                continue
    if len(filters) == 2:
        return mono_webbpsf(waves[0],filters[0]), mono_webbpsf(waves[1],filters[1])
    else:
        #return 0,0 when failed to loacte filter
        return [0,0]

#generate kernel for convolving hb for psf matching
def gen_kernel(psf_hb,psf_ha):
    #generate kernel to match psf_Hb to psf_Ha
        kernel = fits.ImageHDU()
        window = match.CosineBellWindow(alpha=1.25)
        kernel.data = match.create_matching_kernel(psf_hb.data,psf_ha.data,window=window)
        kernel.name = 'PSF_MATCH'
        return kernel

#convolve hb line maps to match psf of ha
def psf_convolve(hb,kernel):
    return fits.ImageHDU(
        data = convolve_fft(hb.data,kernel.data),
        header = hb.header,
        name = 'LINE_HB_CONVOLVED')

### process datasets

In [4]:
#list of obj selected 
obj_lis = Table.read('spectra-fitting_selected.fits')
row_to_remove = []

step = 0 #every 10 steps clear garbage 

for index in tqdm(range(len(obj_lis)),
                desc="Processing Table",
                ncols=100):

    obj=obj_lis[index]
    #if os.path.exists(file_path(obj=obj,prefix='extracted')):
    #    continue

    path = file_path(obj,'full')
    with fits.open(path) as hdu:
    
        #extract HaHb from full data
        extracted = exrtract_HaHb(hdu)
        
        #calculate monochromatic psf
        #only proceed if psf is correctly calculated           
        psf_ha,psf_hb =calc_psf(extracted[0].header['redshift'])
        if psf_ha != 0 and psf_hb !=0:
            psf_ha.name = 'PSF_HA'
            psf_hb.name = 'PSF_HB'

            #generate kernel and convolve
            kernel = gen_kernel(psf_hb,psf_ha)
            hb_convolve =  psf_convolve(extracted[6],kernel)
            #also convolve hb error map
            hb_err = 1/np.sqrt(extracted[7].data)
            hb_weight_convolved = fits.ImageHDU(data = convolve_fft(extracted[7].data,kernel.data),
                                            header=hdu[7].header,
                                            name='LINEWHT_HB_convolved')
            
            extracted.append(psf_ha)
            extracted.append(psf_hb)
            extracted.append(kernel)
            extracted.append(hb_convolve) 
            extracted.append(hb_weight_convolved)

            rewrite = file_path(obj=obj,prefix='extracted')
            extracted.writeto(rewrite,overwrite=True)
                
        else:
            row_to_remove.append(index)   
        step += 1
        if step >= 10:
            gc.collect()   
            step=0 

print('number of objects before psf matching',len(obj_lis))
#remove rows with unsuccessful psf matching
obj_lis.remove_rows(row_to_remove)
print('number of objects after  psf matching',len(obj_lis))
obj_lis.write('spectra-fitting_selected_psfmatched.fits',overwrite=True)

Processing Table: 100%|███████████████████████████████████████████| 752/752 [11:14<00:00,  1.11it/s]


number of objects before psf matching 752
number of objects after  psf matching 692
